<a href="https://colab.research.google.com/github/Zaheer505/Tiny-ML/blob/main/tf_lite_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import tensorflow as tf
import h5py
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof

In [5]:
# Must check this because tf 1.x is different and has different rules.
print(tf.__version__)

2.6.0


## Getting Data 

In [6]:
# getting MNIST Fashion data set
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 3s 1us/step


In [7]:
# Name of classes in our data set
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Exploring the data


In [8]:
print(len(train_labels) )
print(train_images.shape)
print(train_labels)
print(test_images.shape)
print(len(test_labels))

60000
(60000, 28, 28)
[9 0 0 ... 3 0 5]
(10000, 28, 28)
10000


## Setting up Data

In [9]:
# Converting data into Binary Images
train_images = train_images / 255.0
test_images = test_images / 255.0

## Model Defination

In [10]:
# Neural Network Model
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10)
])

In [11]:
# Optimizer, loss and metric to know accuracy 
model.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
# to know what our NN model look like, we can get it's summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


## Training

In [13]:
# Fitting images to Model
# Epoch = number of times that the learning algorithm will work through the entire training dataset
# More Epochs result in more accuracy but can result in overfitting
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4789 - accuracy: 0.8273
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3576 - accuracy: 0.8685
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3261 - accuracy: 0.8798
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3025 - accuracy: 0.8883
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2831 - accuracy: 0.8949
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2692 - accuracy: 0.8986
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2553 - accuracy: 0.9033
Epoch 8/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2449 - accuracy: 0.9074
Epoch 9/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2313 - accuracy: 0.9128
Epoch 10/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.226

## Testing Model

In [15]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', round(test_acc*100, 2), '%')

313/313 - 0s - loss: 0.3707 - accuracy: 0.8760

Test accuracy: 87.6 %


## Saving Trained Model

In [16]:
export_dir = 'saved_model/2'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/2\assets


In [ ]:
print ('Size:', round(os.path.getsize(tf_model_name)/(1024*1024), 3) , 'MB')

Size: 1.589 MB


## Now Converting into TensforFlow Lite Model ( luqman )


In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [19]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

546228

In [20]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_flatten_input:0', 'index': 0, 'shape': array([ 1, 28, 28]), 'shape_signature': array([-1, 28, 28]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 11, 'shape': array([ 1, 10]), 'shape_signature': array([-1, 10]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [21]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]


ValueError: Cannot set tensor: Dimension mismatch. Got 2 but expected 3 for input 0.

## Zaheer's Implementation of TF Lite 

In [ ]:
tf_lite_model_file_name = "tf_lite_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] # optimizing for size 
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpwri52zf2/assets


INFO:tensorflow:Assets written to: /tmp/tmpwri52zf2/assets


In [ ]:
tflite_model_name = tf_lite_model_file_name
open(tflite_model_name, "wb").write(tflite_model)

139408

In [ ]:
print ('Size:', round(os.path.getsize(tf_lite_model_file_name)/(1024), 3) , 'KB')

Size: 136.141 KB


**Input Shape Check**

In [ ]:
interpreter = tf.lite.Interpreter(model_path = tf_lite_model_file_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

**Resizing Shape**

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
test_images.dtype # checking orignal data type

dtype('float64')

In [ ]:
test_imgs_numpy = np.array(test_images, dtype=np.float32) # convert to new data type

**Prediction/Accuracy Check**

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
tf_prediction_classes = np.argmax(tflite_model_predictions, axis=1)
print (tf_prediction_classes)

Prediction results shape: (10000, 10)
[9 2 1 ... 8 1 5]


In [ ]:
acc = accuracy_score(tf_prediction_classes, test_labels)
print ('Accuracy:', round(acc*100, 4), '%')

Accuracy 87.86 %
